### Convolutional Neural Network
**Convolutional Neural Network (CNN)** is a type of `deep` learning model designed to process data with a grid-like structure, such as images, by using layers that automatically and adaptively learn spatial hierarchies of features. It consists of convolutional layers that apply filters to extract features, pooling layers that reduce spatial dimensions, and fully connected layers for classification. CNNs are widely used in image recognition, object detection, and computer vision tasks due to their efficiency and accuracy.


Our task is to build a CNN model capable of correctly classifying whether an image is of the four labels:
- `tom`- only contains tom
- `jerry`-only contains jerry
- `tom_jerry_1` contains both tom and jerry
- `tom_jerry_0` contains none of them

In [ ]:
#import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

##keras models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define base directory
base_dir = 'tj'

# Check subfolders (should be: tom, jerry, tom_jerry_1, tom_jerry_0)
print("Subfolders in the dataset directory:")
print(os.listdir(base_dir))

In [ ]:
#loading ground_truth CSV
ground_truth_df = pd.read_csv('./csv_data/ground_truth.csv')
print("\nGround truth sample:")
ground_truth_df.tail()


Before feeding images into a neural network, we need to know:

- What size they are (height × width × channels)

- Whether they're consistent in shape

- Whether they need resizing

CNNs require fixed-size inputs, so this step helps us decide on a common input shape (e.g., **64×64** or **128×128**). We’ll do this randomly by sampling a few images from different folders and inspect them.

In [ ]:
from PIL import Image
import random

folders = ['tom', 'jerry', 'tom_jerry_1', 'tom_jerry_0']

#dictionary to hold our samples
samples = {}

for folder in folders:
    folder_path = os.path.join(base_dir, folder)
    files = os.listdir(folder_path)

    if files:
        image = random.choice(files)
        img_path = os.path.join(folder_path, image)
        img = Image.open(img_path)

        samples[folder] = (img, img_path)


In [ ]:
#now lets display the image shapes
for label, (img, path) in samples.items():
    #lin to return the image size
    print(f'{label.upper()} image shape: {img.size} - {os.path.basename(path)}')
    plt.imshow(img)
    plt.title(f'Category: {label}')
    plt.axis('off')
    plt.show()

## Data Preprocessing
### Resizing the images
Before we move to modelling, we need to preprocess and adjust our images. The most common steps are `resizing` and `flattening`.

**Resizing** maintains the sructure of the image but standardizes its size for efficient processing.

**Flattening** converts a multi-dimensional array/image into a vector(1D). This helps us in changing a 3D tensor into a 1D tensor for compatibility with dense layers in Neural networks.

In [ ]:
#we'll use imagedatagenerator to resize 
size = (64, 64)
batch_size = 40 #the number of images to be processed at a time

#normalizing the data(think of this as similar step to using StandardScaler)
normalize = ImageDataGenerator(rescale=1./255, validation_split=0.3)

#create training set and validation set
train_set = normalize.flow_from_directory(
    base_dir, 
    target_size = size,
    batch_size = batch_size,
    class_mode = 'categorical',
    subset='training',
    shuffle = True
    # color_mode =  'grayscale' #uncomment this line for faster runtime
)

#validation set 
validation_set = normalize.flow_from_directory(
    base_dir, 
    target_size = size,
    batch_size = batch_size,
    class_mode = 'categorical',
    subset='validation',
    shuffle = True
    # color_mode =  'grayscale' #uncomment this line for faster runtime
)

- The `rescale=1.255` is to normalize the pixel values, which were initially RGB(0, 255) to (0, 1).

- `flow_from_directory` method from ImageDataGenerator has a special requirement that each of the different classes/labels we are modelling for be in its own subfolder.

Now we can move into building our Convolutional Neural Network and flatten the image before passing it into our dense layers.


In [ ]:
##designing our CNN and dense layers
model = Sequential()

#CNN layer 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))


##CNN layer 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))


##flattening before pushing to dense layers
model.add(Flatten())

""" 
Design three hidden layers
First one with 128 neurons
Second one with 64 neurons
Third one with 32 neurons

Use Dropout technique on the first two with your desired percentage

Also include the output layer, choose the correct number of neurons and activation function

"""

#write your code here

model.summary() #this line give you a table-like representation of your design

We'll now train our CNN using the training and validation datasets.


`PS:` The CNN model extracts different features from the images then our dense layer is used to interpret them and classify the image. 

In [ ]:
"""
during training make sure to pick a small epoch to reduce run-time
replace ... with a valid number
"""
model.fit(
    train_set, 
    epochs=...,
    validation_data = validation_set
)

### Evaluation 
Lets check how the model performs on the validation set.

Here are tables to help you:
| Validation Loss | Notes                            |
| --------------- | -------------------------------- |
| > 1.0           | Model struggling or too simple   |
| 0.8 – 1.0       | May still be improving           |
| 0.5 – 0.8       | Acceptable, improving            |
| 0.2 – 0.5       | Good model performance           |
| < 0.2           | Excellent, but check for overfit |

And an accuracy table

| Accuracy Range | Interpretation                                  |
| -------------- | ----------------------------------------------- |
| < 60%          | Likely underfitting or data issues              |
| 60–75%         | Learning, but room for model/data improvement   |
| 75–85%         | Good starting point, acceptable for class demos |
| 85–95%         | Strong, usable for screen time analysis         |
| > 95%          | Excellent, but check for overfitting            |


In [ ]:
# Evaluate on validation data
val_loss, val_accuracy = model.evaluate(validation_set)
print(f"\nValidation Accuracy: {val_accuracy:.2f}")
print(f"Validation Loss: {val_loss:.4f}")


###  Prediction 
Now we need to create a helper function to utilize our model in prediction tasks.

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

"""
fill in the logic of the function:
1. load and preprocess the image
2. use the model to predict
3. Display the image and the prediction
"""
# Mapping from class index to label
class_labels = list(train_set.class_indices.keys())

def predict_and_render(image_path):

    #write your code here
    pass


In [ ]:
""" 
Testing your function on a random image
"""

#pick an image path manually
sample_image_path = os.path.join(base_dir, 'tom_jerry_1', random.choice(os.listdir(os.path.join(base_dir, 'tom_jerry_1'))))

#predict
predict_and_render(sample_image_path)
